In [1]:
import folium
from folium import Element
import pandas as pd
from string import Template

In [2]:
df = pd.read_csv("log_sheet.csv").fillna("").astype(str)

In [3]:
df = df.rename(columns = {"Name*": "Name"})

for col in ["Metres", "Matthew"]:
    df[col] = df[col].apply(lambda x: x.split(".")[0])

In [4]:
df

,Name,Region (SBC The Munros),County,Metres,Feet,Latitude,Longitude,Matthew,Scott,More folk...,...,....1,Unnamed: 12
0,Ben Nevis [Beinn Nibheis],04A: Fort William to Loch Treig & Loch Leven,Highland,1344,4411,56.796891,-5.003675,,2005.0,,,,"*n.b.\nThere are various name conventions, so ..."
1,Ben Macdui [Beinn Macduibh],08A: Cairngorms,Aberdeenshire/Moray,1309,4296,57.070368,-3.669099,2024,,,,,
2,Braeriach,08A: Cairngorms,Aberdeenshire/Highland,1296,4252,57.078298,-3.728373,,,,,,
3,Cairn Toul,08A: Cairngorms,Aberdeenshire,1291,4236,57.054406,-3.710757,,,,,,
4,Sgor an Lochain Uaine,08A: Cairngorms,Aberdeenshire,1258,4127,57.058376,-3.725897,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Meall na Teanga,10C: Loch Arkaig to Glen Moriston,Highland,916,3008,56.98903,-4.93094,,,,,,
278,Beinn a' Chleibh,01D: Inveraray to Crianlarich,Argyll and Bute/Stirling,916,3006,56.39023,-4.83563,,,,,,
279,Ben Vane,01D: Inveraray to Crianlarich,Argyll and Bute,915,3004,56.249786,-4.781655,,,,,,
280,Carn Aosda,06B: Pitlochry to Braemar & Blairgowrie,Aberdeenshire,915,3003,56.895696,-3.423274,2024,,,,,


In [5]:
table_css = """<style>
              table {
            
                    border-collapse: collapse;
                    font-family: Arial, sans-serif;
                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                    background: #fff;
                    table-layout: fixed;
                  }

                    th {
                        border: 1px solid #ddd;
                        background-color: #f7f7f7;
                        text-align: left;
                        font-weight: 600;
                        height: 70px;
                      }
            
                td {
                    border: 1px solid #ddd;
                    text-align: left;
                    height: 70px;
                  }

            </style>"""

In [6]:
table_search = """
<script>
  document.addEventListener("DOMContentLoaded", function () {
    const searchInput = document.getElementById('searchInput');
    const rows = document.querySelectorAll('#mountainTable tbody tr');
    searchInput.addEventListener('input', function () {
      const filter = searchInput.value.toLowerCase();
      rows.forEach(row => {
        const nameText = row.cells[0].textContent.toLowerCase();
        row.style.display = nameText.includes(filter) ? '' : 'none';
      });
    });
  });
</script>
"""

In [7]:
row_template = Template("""<tr><td><button onclick="window.map.setView([$lat, $long], 13)">$name</button></td></tr>""")

In [8]:
table_rows = ""
for _, row in df.iterrows():

    table_rows += row_template.substitute({'name': row["Name"],
                            'lat': row["Latitude"],
                            'long': row["Longitude"],
                            })


In [9]:
table_css = """
<style>
  table {
    border-collapse: collapse;
    font-family: Arial, sans-serif;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    background: #fff;
    table-layout: fixed;
  }
  th {
    border: 1px solid #ddd;
    background-color: #f7f7f7;
    text-align: left;
    font-weight: 600;
    height: auto;
  }
  td {
    border: 1px solid #ddd;
    text-align: left;
    height: auto;
  }
</style>
"""

table_script = """
<script>
  document.addEventListener("DOMContentLoaded", function () {
    const searchInput = document.getElementById('searchInput');
    const rows = document.querySelectorAll('#mountainTable tbody tr');
    searchInput.addEventListener('input', function () {
      const filter = searchInput.value.toLowerCase();
      rows.forEach(row => {
        const nameText = row.cells[0].textContent.toLowerCase();
        row.style.display = nameText.includes(filter) ? '' : 'none';
      });
    });
  });
</script>
"""

sidebar_html = f"""
            <div id="sidebar" style="
              position: fixed;
              top: 0;
              left: 0;
              width: 170px;
              height: 100vh;
              background: #ffffff;
              overflow-y: auto;
              padding: 20px 15px;
              box-shadow: 2px 0 10px rgba(0, 0, 0, 0.1);
              z-index: 1000;
              font-family: 'Segoe UI', Tahoma, sans-serif;
              font-size: 14px;
              line-height: 1.6;
              color: #333;
              border-right: 1px solid #ddd;
              transition: transform 0.3s ease;
            ">
            <p style="font-weight:bold"> {len(df[df["Matthew"] != ""])}/282 bagged</p>
        
              <table id="mountainTable">
                <colgroup><col style="width: 100px"></colgroup>
                <thead>
                  <tr>
                    <th>
                      Name<br>
                      <input type="text" id="searchInput" placeholder="🔍" style="width: 100px">
                    </th>
                  </tr>
                </thead>
                <tbody>
                  {table_rows}
                </tbody>
              </table>
            </div>
           

<!-- Toggle Button -->
    <button id="toggleSidebar" style="
      position: fixed;
      top: 10px;
      left: 180px;
      z-index: 1100;
      padding: 5px 10px;
      background-color: #ffffff;
      border: 1px solid #ccc;
      border-radius: 4px;
      cursor: pointer;
      font-size: 12px;
    ">Hide table</button>

"""

scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6, zoom_control=False)

map_div_id = m.get_name()
map_css = f"""
<style>
  #{map_div_id} {{
    position: absolute;
    top: 0;
    left: 180;
    right: 0;
    bottom: 0;
    z-index: 0;
  }}
</style>
"""

assign_map_js = f"""
<script>
  window.onload = function() {{
    window.map = {map_div_id};
    window.panMap = function(lat, lng) {{
      window.map.setView([lat, lng], 10);
    }};
  }};
</script>
"""

assign_toggle_js = """
<script>
  const sidebar = document.getElementById('sidebar');
  const toggleButton = document.getElementById('toggleSidebar');

  toggleButton.addEventListener('click', function () {
    if (sidebar.style.transform === 'translateX(-170px)') {
      sidebar.style.transform = 'translateX(0)';
      toggleButton.style.left = '180px';
      toggleButton.textContent = 'Hide table';
    } else {
      sidebar.style.transform = 'translateX(-170px)';
      toggleButton.style.left = '10px';
      toggleButton.textContent = 'Show table';
    }
  });
</script>
"""

m.get_root().header.add_child(Element(map_css))
m.get_root().header.add_child(Element(table_css))
m.get_root().html.add_child(Element(sidebar_html))
m.get_root().header.add_child(Element(table_script))
m.get_root().html.add_child(Element(assign_map_js))
m.get_root().html.add_child(Element(assign_toggle_js))

In [10]:
bag = folium.CustomIcon(icon_image="https://raw.githubusercontent.com/mprallison/munro_bagger/main/images/matthew.png",
                        icon_size=(14, 14),
                        icon_anchor=(7, 7)
                        )

gap = folium.CustomIcon(icon_image="https://raw.githubusercontent.com/mprallison/munro_bagger/main/images/peak.png",
                            icon_size=(14, 14),
                            icon_anchor=(7, 7)
                            )

In [11]:
def add_gap(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]

    label = "<br>".join([name, f"height: {height}m"])

    folium.Marker(
                location = coords,
                tooltip = label,
                icon = gap
                ).add_to(m)

    return

In [12]:
def add_bag(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]
    year = row["Matthew"]

    label = "<br>".join([name, f"height: {height}m", f"year: {year}"])
    
    folium.Marker(
    location = coords,
    tooltip = label,
    icon = bag
    ).add_to(m)

    return

In [13]:
#add icons
for _, row in df.iterrows():

    if row["Matthew"] != "":  
        add_bag(row)
    else:
        add_gap(row)

In [14]:
# Save
m.save("index.html")